# Ejercicio: Crear un DataFrame Completo con SWAPI

## Objetivo:
El objetivo de este ejercicio es que los alumnos practiquen la obtención de datos de una API REST y su posterior procesamiento para crear un DataFrame en Pandas que contenga información detallada sobre personajes de Star Wars.

## Instrucciones:

1. Obtener los datos:

- Utilizar la API de Star Wars (SWAPI) para obtener datos sobre los personajes (people) de la saga.
- La URL base para obtener la lista de personajes es: https://swapi.dev/api/people/.
- Dado que los datos están paginados, deberán recorrer todas las páginas para obtener la lista completa de personajes.

2. Crear el DataFrame:

    Crear un DataFrame que incluya las siguientes columnas con la información detallada de cada personaje:

    - id: Identificador único del personaje.
    - name: Nombre del personaje.
    - height: Altura del personaje (en centímetros).
    - mass: Peso del personaje (en kilogramos).
    - hair_color: Color de cabello.
    - skin_color: Color de piel.
    - eye_color: Color de ojos.
    - birth_year: Año de nacimiento.
    - gender: Género.
    - homeworld: Nombre del planeta de origen (obtenido mediante una petición adicional a la API).
    - species: Nombre de la especie del personaje (obtenido mediante una petición adicional si el campo species no está vacío).
    - films: Lista de nombres de películas en las que aparece el personaje (obtenido mediante peticiones adicionales).

3. Procesamiento de datos:

    - Cada entrada en el campo homeworld de los personajes es un enlace a la API, desde el cual se debe obtener el nombre del planeta.

    - El campo species contiene una lista de enlaces. Si no está vacío, se debe obtener el nombre de la especie desde la API.

    - El campo films contiene una lista de enlaces a las películas. Se debe realizar una petición por cada enlace para obtener el título de la película.

4. Entrega del ejercicio:

    - Se debe entregar un archivo .ipynb (Jupyter Notebook) con el código que obtenga los datos, procese las respuestas y construya el DataFrame completo.

    - También deben incluir en el notebook una breve descripción de cómo organizaron el proceso para obtener los datos y construir el DataFrame.

Tips:

- Utilicen la librería requests para realizar las peticiones HTTP.
- Tengan en cuenta que la API utiliza paginación, por lo que deberán iterar sobre las páginas hasta obtener todos los datos.
- Los datos relacionados (homeworld, species, films) requieren realizar peticiones adicionales, así que gestionen adecuadamente los tiempos de espera (pueden considerar utilizar la función time.sleep() para evitar sobrecargar la API).
- Si encuentran campos con datos faltantes o errores, documenten cómo lo manejan en su notebook.

In [1]:
import hashlib
import requests
import datetime
import pandas as pd
from IPython.display import HTML
from requests import HTTPError
import time

In [28]:
#('https://swapi.py4e.com/')


In [4]:
import requests
import pandas as pd

# Función para obtener todos los personajes
def obtener_personajes():
    url = "https://swapi.py4e.com/api/people/"
    personajes = []
    
    while url:
        response = requests.get(url)
        data = response.json()
        personajes.extend(data['results'])
        url = data['next']  # Paginación
        
    return personajes

# Obtener todos los personajes
personajes = obtener_personajes()


Creo el dataFrame

In [8]:
import requests
import pandas as pd

# Obtener los datos de personajes (mismo paso que antes)
def obtener_personajes():
    url = "https://swapi.py4e.com/api/people/"
    personajes = []
    
    while url:
        response = requests.get(url)
        data = response.json()
        personajes.extend(data['results'])
        url = data['next']  # Paginación
        
    return personajes

personajes = obtener_personajes()

# Crear un DataFrame vacío
columnas = [
    'id', 'name', 'height', 'mass', 'hair_color', 'skin_color',
    'eye_color', 'birth_year', 'gender', 'homeworld', 'species', 'films'
]
df_personajes = pd.DataFrame(columns=columnas)

# Crear una lista para almacenar las filas
filas = []

# Rellenar la lista con la información de los personajes
for idx, personaje in enumerate(personajes):
    filas.append({
        'id': idx + 1,
        'name': personaje['name'],
        'height': personaje['height'],
        'mass': personaje['mass'],
        'hair_color': personaje['hair_color'],
        'skin_color': personaje['skin_color'],
        'eye_color': personaje['eye_color'],
        'birth_year': personaje['birth_year'],
        'gender': personaje['gender'],
        'homeworld': personaje['homeworld'],
        'species': personaje['species'],
        'films': personaje['films']
    })

# Convertir la lista a un DataFrame
df_personajes = pd.DataFrame(filas)


Procesamiento de datos:

In [11]:
# Función para obtener el nombre del planeta
def obtener_homeworld(url):
    response = requests.get(url)
    time.sleep(1)  # Pausa de 1 segundo
    return response.json()['name']

# Función para obtener el nombre de la especie
def obtener_especies(urls):
    especies = []
    for url in urls:
        response = requests.get(url)
        time.sleep(1)  # Pausa de 1 segundo
        especies.append(response.json()['name'])
    return especies

# Función para obtener los títulos de las películas
def obtener_films(urls):
    films = []
    for url in urls:
        response = requests.get(url)
        time.sleep(1)  # Pausa de 1 segundo
        films.append(response.json()['title'])
    return films


In [10]:
# Función para obtener el nombre del planeta
def obtener_homeworld(url):
    response = requests.get(url)
    time.sleep(1)  # Pausa de 1 segundo
    return response.json()['name']

# Función para obtener el nombre de la especie
def obtener_especies(urls):
    especies = []
    for url in urls:
        response = requests.get(url)
        time.sleep(1)  # Pausa de 1 segundo
        especies.append(response.json()['name'])
    return especies

# Función para obtener los títulos de las películas
def obtener_films(urls):
    films = []
    for url in urls:
        response = requests.get(url)
        time.sleep(1)  # Pausa de 1 segundo
        films.append(response.json()['title'])
    return films

# Procesar cada personaje
for index, row in df_personajes.iterrows():
    # Obtener el nombre del planeta
    row['homeworld'] = obtener_homeworld(row['homeworld'])
    
    # Obtener el nombre de las especies
    if row['species']:
        row['species'] = obtener_especies(row['species'])
    else:
        row['species'] = []

    # Obtener los títulos de las películas
    row['films'] = obtener_films(row['films'])

# Actualizar el DataFrame
df_personajes.update(df_personajes)
df_personajes


,id,name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,homeworld,species,films
0,1,Luke Skywalker,172,77,blond,fair,blue,19BBY,male,https://swapi.py4e.com/api/planets/1/,[https://swapi.py4e.com/api/species/1/],"[https://swapi.py4e.com/api/films/1/, https://..."
1,2,C-3PO,167,75,n/a,gold,yellow,112BBY,n/a,https://swapi.py4e.com/api/planets/1/,[https://swapi.py4e.com/api/species/2/],"[https://swapi.py4e.com/api/films/1/, https://..."
2,3,R2-D2,96,32,n/a,"white, blue",red,33BBY,n/a,https://swapi.py4e.com/api/planets/8/,[https://swapi.py4e.com/api/species/2/],"[https://swapi.py4e.com/api/films/1/, https://..."
3,4,Darth Vader,202,136,none,white,yellow,41.9BBY,male,https://swapi.py4e.com/api/planets/1/,[https://swapi.py4e.com/api/species/1/],"[https://swapi.py4e.com/api/films/1/, https://..."
4,5,Leia Organa,150,49,brown,light,brown,19BBY,female,https://swapi.py4e.com/api/planets/2/,[https://swapi.py4e.com/api/species/1/],"[https://swapi.py4e.com/api/films/1/, https://..."
...,...,...,...,...,...,...,...,...,...,...,...,...
82,83,Finn,unknown,unknown,black,dark,dark,unknown,male,https://swapi.py4e.com/api/planets/28/,[https://swapi.py4e.com/api/species/1/],[https://swapi.py4e.com/api/films/7/]
83,84,Rey,unknown,unknown,brown,light,hazel,unknown,female,https://swapi.py4e.com/api/planets/28/,[https://swapi.py4e.com/api/species/1/],[https://swapi.py4e.com/api/films/7/]
84,85,Poe Dameron,unknown,unknown,brown,light,brown,unknown,male,https://swapi.py4e.com/api/planets/28/,[https://swapi.py4e.com/api/species/1/],[https://swapi.py4e.com/api/films/7/]
85,86,BB8,none,unknown,none,none,black,unknown,none,https://swapi.py4e.com/api/planets/28/,[https://swapi.py4e.com/api/species/2/],[https://swapi.py4e.com/api/films/7/]
